In [ ]:
import tensorflow as tf 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
iris_data = pd.read_csv ('iris.csv')

In [ ]:
iris_data ['variety'].unique()

In [ ]:
setosa_data = iris_data[iris_data['variety']=='Setosa']
versicolor_data = iris_data[iris_data['variety']=='Versicolor']
virginica_data = iris_data[iris_data['variety']=='Virginica']

In [ ]:
setosa_data.head()

In [ ]:
versicolor_data.head()

In [ ]:
virginica_data.head()

In [ ]:
x_setosa = setosa_data ['sepal.length']
y_setosa = setosa_data ['sepal.width']

x_versicolor = versicolor_data ['sepal.length']
y_versicolor = versicolor_data ['sepal.width']

x_virginica = virginica_data ['sepal.length']
y_virginica = virginica_data ['sepal.width']

In [ ]:
plt.scatter(x_setosa, y_setosa)
plt.scatter(x_versicolor, y_versicolor, marker ='*')
plt.scatter(x_virginica, y_virginica, marker='P')
plt.xlabel('sepal length', fontsize=15)
plt.ylabel ('sepal width', fontsize=15)

In [ ]:
x_setosa_p = setosa_data ['petal.length']
y_setosa_p = setosa_data ['petal.width']

x_versicolor_p = versicolor_data ['petal.length']
y_versicolor_p = versicolor_data ['petal.width']

x_virginica_p = virginica_data ['petal.length']
y_virginica_p = virginica_data ['petal.width']

plt.scatter(x_setosa_p, y_setosa_p)
plt.scatter(x_versicolor_p, y_versicolor_p, marker ='*')
plt.scatter(x_virginica_p, y_virginica_p, marker='P')
plt.xlabel('petal length', fontsize=15)
plt.ylabel ('petla width', fontsize=15)

Versicolor is very similar to the virginica, so let's drop virginica

In [ ]:
x_setosa = setosa_data ['sepal.length']
y_setosa = setosa_data ['sepal.width']

x_versicolor = versicolor_data ['sepal.length']
y_versicolor = versicolor_data ['sepal.width']

x_virginica = virginica_data ['sepal.length']
y_virginica = virginica_data ['sepal.width']

plt.scatter(x_setosa, y_setosa)
plt.scatter(x_versicolor, y_versicolor, marker ='*')
# plt.scatter(x_virginica, y_virginica, marker='P')
plt.xlabel('sepal length', fontsize=15)
plt.ylabel ('sepal width', fontsize=15)

Prepare the data

In [ ]:
X_Setosa = setosa_data[['sepal.length', 'sepal.width']]
X_Versicolor = versicolor_data[['sepal.length', 'sepal.width']]

X_Setosa.loc[:,'label'] = X_Setosa.shape[0]*[1]
X_Versicolor.loc[:,'label'] = X_Versicolor.shape[0]*[0]

In [ ]:
X = pd.concat([X_Setosa, X_Versicolor])
X = X.sample(frac = 1)

X_train = X[:80]
X_eval = X[80:]

y_train = X_train ['label']
X_train = X_train.drop('label', axis =1)
y_eval = X_eval['label']
X_eval = X_eval.drop('label', axis =1)

Build the model

In [ ]:
feature_col_1 = tf.feature_column.numeric_column(key='sepal.length')
feature_col_2 = tf.feature_column.numeric_column(key='sepal.width')

linear_classifier = tf.estimator.LinearClassifier(feature_columns=[feature_col_1, feature_col_2])

In [ ]:
def make_input_fn(data, labes, batch_size=30, num_epochs=6000):
    def input_fn():
        ds = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train)) 
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_fn

input_fn = make_input_fn(X_train, y_train)
linear_classifier.train(input_fn)

In [ ]:
eval_input_fn = make_input_fn(X_eval, y_eval, num_epochs=1)
linear_classifier.evaluate(eval_input_fn)

Visualizing the results in 3D

In [ ]:
linear_classifier.get_variable_names()

In [ ]:
b =linear_classifier.get_variable_value('linear/linear_model/bias_weights')
w1 =linear_classifier.get_variable_value('linear/linear_model/sepal.length/weights')
w2 =linear_classifier.get_variable_value('linear/linear_model/sepal.width/weights')


In [ ]:
x11 = 4.0
x21 = (-w1*x11-b)/w2
x12 = 8.5 
x22 = (-w1*x12-b)/w2
# print(type(x21), ' ', type(x22))
# print(x21, ' ', x22)
x21 = x21[0][0]
x22 = x22[0][0]
# print(type(x21), ' ', type(x22))

In [ ]:
plt.scatter(x_setosa, y_setosa)
plt.scatter(x_versicolor, y_versicolor, marker ='*')
plt.xlabel('sepal length', fontsize=15)
plt.ylabel ('sepal width', fontsize=15)
plt.plot ([x11,x12],[x21,x22], 'm-')
x_array = np.arange(x11, x12, 1)
y_array = np.arange(x21, x22, 1)
xx, yy = np.meshgrid(x_array, y_array)
# zz = np.zeros((xx.shape[0], xx.shape[1]))
zz = w1*xx +w2*yy +b




In [ ]:
%matplotlib notebook
plt3d = plt.figure().gca(projection='3d')
plt3d.scatter (x_setosa, y_setosa, len(y_setosa)*[0], marker='o')
plt3d.scatter (x_versicolor, y_versicolor, len(x_versicolor)*[0], marker='*')
plt3d.plot ([x11,x12],[x21,x22],[0,0], 'm-')
plt3d.scatter (xx,yy,zz)
plt3d.plot_surface(xx, yy, zz, alpha=0.3, color ='k')


Materials from AIGenius mini course Linear Classification with Tensorflow2.0 provided by Dr. Thomas Vanck 